# Assignment 3: NLP

## Problem 1. The Markowitz Problem

In [8]:
import gurobipy as gb
import numpy as np
from gurobipy import *

In [2]:
assets = ["FTSE 100", "DAX", "DJIA", "DJ Asian Titans 50", "Russell 2000"]
return_anu = [0.00069, 0.00640,  0.00693, -0.00108, 0.00688]
return_std = [3.23, 5.25, 3.26, 4.49, 4.60]

corr_matrix = [[1 , 0.75526686, 0.77283301, 0.68746749, 0.64982921],
               [0.75526686, 1 , 0.70381606, 0.66804974, 0.69425926],
               [0.77283301, 0.70381606, 1 , 0.72145449, 0.8484989 ],
               [0.68746749, 0.66804974, 0.72145449, 1 , 0.67030134],
               [0.64982921, 0.69425926, 0.8484989 , 0.67030134, 1 ]]

# See Excel File for more details and formulas on how the above values were found

#### (a) Formulate an NLP to devise the optimal portfolio that minimizes the portfolio risk subject to non-negative return. What are the optimal solution and value?
#### Hint: Notice that for the stock FTSE 100, for example, the rate of return from May 3 to June 1
#### 2016 is (5,996.80 – 6230.80)/6230.80 = -0.0424.

In [3]:
model2 = gb.Model("Portfolio Optimization Minimize Risk part a")

L = len(assets)

# Variables:
A = model2.addVars(L, vtype = GRB.CONTINUOUS, name = ["Asset_"+assets[i] for i in range(L)])

# Set Objective:
risk = sum(A[i]*return_std[i] * A[j]*return_std[j] *corr_matrix[i][j] for i in range(L) for j in range(L))
model2.setObjective(risk, GRB.MINIMIZE)

# Constraints:
model2.addConstr(sum(A[i] for i in range(L))==1, name = "Fraction Cosntraint")
model2.addConstr(sum(A[i]*return_anu[i] for i in range(L))>=0)

model2.optimize()

print("The minimimal portfolio variance is {}".format(round(model2.objVal**(1/2),2)))

print("\nThe optimal allocation is:")
for v in model2.getVars():
    print(v.varName, "=",round(v.x,3))

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-10
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 2 rows, 5 columns and 10 nonzeros
Model fingerprint: 0x904faf10
Model has 15 quadratic objective terms
Coefficient statistics:
  Matrix range     [7e-04, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+01, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.06s
Presolved: 2 rows, 5 columns, 10 nonzeros
Presolved model has 15 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 4
 AA' NZ     : 1.500e+01
 Factor NZ  : 2.100e+01
 Factor Ops : 9.100e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.39436272e+05 -1.39436272e

#### (b) Assume the initial allocation is 20% in each index. Changing the position requires incurring transaction costs. Formulate an NLP to find the optimal portfolio to minimize risk subject to non-negative return. What are the optimal solution and value?

In [4]:
model = gb.Model("Portfolio Optimization Minimize Risk part b")

L = len(assets)

# Variables:
A = model.addVars(L, vtype = GRB.CONTINUOUS, name = ["Asset_"+assets[i] for i in range(L)])

# Set Objective:
risk = sum(A[i]*return_std[i] * A[j]*return_std[j] *corr_matrix[i][j] for i in range(L) for j in range(L))
model.setObjective(risk, GRB.MINIMIZE)

# Constraints:
model.addConstr(sum(A[i] for i in range(L))==1, name = "Fraction Constraint")
## Constraint for cost of changing postfolio position:
model.addConstr(sum(A[i]*return_anu[i] for i in range(L))-sum((A[i]-0.2)**2 for i in range(L))>=0)

model.optimize()

print("The minimimal portfolio variance is {}".format(round(model.objVal**(1/2),2)))

print("\nThe optimal allocation is:")
for v in model.getVars():
    print(v.varName, "=",round(v.x,3))

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1 rows, 5 columns and 5 nonzeros
Model fingerprint: 0x0c95b170
Model has 15 quadratic objective terms
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [4e-01, 4e-01]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+01, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
  QRHS range       [2e-01, 2e-01]
Presolve time: 0.17s
Presolved: 14 rows, 14 columns, 40 nonzeros
Presolved model has 2 second-order cone constraints
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 8.300e+01
 Factor NZ  : 1.050e+02
 Factor Ops : 1.015e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
  

## Problem 2: The CYCOM Corporation Problem

In [5]:
projects = ["project 1", "project 2", "project 3", "project 4", "project 5", "project 6"]
p = [1.1, 0.5, 2.5, 1.6, 2.2, 2.4]
startup_cost = [325, 200, 490, 125, 710, 240]
profit_if_successful = [1750, 700, 1300, 800, 1450, 1300]
total_eng = 25
cost_eng = 150000
P = len(projects)

#### (a) Construct and solve a nonlinear optimization model to determine the number of engineers to assign to each project that will maximize the expected contribution to profit of the six projects minus the cost of assigning engineers to the projects and the start-up costs of the projects. What is the optimal solution? (Rounded to one decimal place)

In [6]:
# Model:
model = gb.Model("CYCOM Corporation Problem part a")

# Decision Variables:
## Number of Engineers per Project:
X = model.addVars(P, vtype = GRB.CONTINUOUS, name = ["Number of engineers assigned to "+projects[i] for i in range(P)])
## Probabilities of success per project:
Prob = model.addVars(P, vtype = GRB.CONTINUOUS, lb = 0, ub = 1, name = ["Probability of success for "+projects[i] for i in range(P)])

# Constraints:
model.addConstrs(Prob[i]*(X[i]+p[i]) == X[i] for i in range(P))
model.addConstr(sum(X[i] for i in range(P))<=25)

# Objective function:
objFunc = sum(Prob[i]*profit_if_successful[i] - (1-Prob[i])*startup_cost[i] - 150*X[i] for i in range(P))
model.setObjective(objFunc, GRB.MAXIMIZE)

model.params.NonConvex = 2
model.optimize()

print("The total profit from realizing the following projects is {}".format(round(model.objVal*1000,1)),"$")

print("\nThe optimal engineers to projects allocation and probabilities of success are:")
for v in model.getVars():
    print(v.varName, "=",round(v.x,3))

Set parameter NonConvex to value 2
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1 rows, 12 columns and 6 nonzeros
Model fingerprint: 0xeee9c90e
Model has 6 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [5e-01, 2e+00]
  Objective range  [2e+02, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 2e+01]

Continuous model is non-convex -- solving as a MIP

Found heuristic solution: objective -2090.000000
Presolve time: 0.00s
Presolved: 25 rows, 12 columns, 42 nonzeros
Presolved model has 6 bilinear constraint(s)
Variable types: 12 continuous, 0 integer (0 binary)

Root relaxation: objective 4.791275e+03, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd 

#### (b) Construct and solve an optimization model that minimizes the standard deviation of the contribution to profit subject to the constraint that the expected contribution to profit (minus the cost of engineers) is at least 1.1 million. What is the optimal solution? (Rounded to one decimal place)

In [9]:
# Model:
model = gb.Model("CYCOM Corporation Problem part b")

# Decision Variables:
## Number of Engineers per Project:
X = model.addVars(P, vtype = GRB.CONTINUOUS, name = ["Number of engineers assigned to "+projects[i] for i in range(P)])
## Probabilities of success per project:
Prob = model.addVars(P, vtype = GRB.CONTINUOUS, lb = 0, ub = 1, name = ["Probability of success for "+projects[i] for i in range(P)])

# Constraints:
model.addConstr(sum(X[i] for i in range (P))<=25)
model.addConstrs(X[i]==(X[i]+p[i])*Prob[i] for i in range(P)) 
model.addConstr(sum(Prob[i]*profit_if_successful[i]-(1-Prob[i])*startup_cost[i]-150*X[i] for i in range(P))>=1100)

# Objective function:
model.setObjective(sum((profit_if_successful[i]+startup_cost[i])**2*Prob[i]*(1-Prob[i]) for i in range(P)),GRB.MINIMIZE)


model.params.NonConvex=2
model.Params.LogToConsole=0

model.optimize()


print ('The minimum variance',round(model.objval,1),'\n')
print ('The minimum standard deviation is ',round(np.sqrt(model.objval),1),'\n')
print('The number of engineers assigned to each projet is ')
for v in model.getVars():
    print(v.varname,'=',round(v.x,1))


Set parameter NonConvex to value 2
The minimum variance 3241282.2 

The minimum standard deviation is  1800.4 

The number of engineers assigned to each projet is 
Number of engineers assigned to project 1 = 5.1
Number of engineers assigned to project 2 = 1.6
Number of engineers assigned to project 3 = 4.2
Number of engineers assigned to project 4 = 1.5
Number of engineers assigned to project 5 = 6.0
Number of engineers assigned to project 6 = 3.0
Probability of success for project 1 = 0.8
Probability of success for project 2 = 0.8
Probability of success for project 3 = 0.6
Probability of success for project 4 = 0.5
Probability of success for project 5 = 0.7
Probability of success for project 6 = 0.6
